Trabalho Otimização


Alunos:
DIOGO EMANUEL ANTUNES SANTOS
PEDRO HENRIQUE ROCHA DE CASTRO


In [857]:
import random
import numpy as np
import matplotlib.pyplot as plt
import time
import json

Definindo a distância entre as cidades

In [858]:
file_name = 'matrix.json'
def create_distance_matrix():
    with open(file_name, 'r') as file:
        matrix = json.load(file)
        return matrix

Função de avaliação (fitness)

In [859]:
def calculate_fitness(solution, distance_matrix, base_city):
    total_distance = 0
    for route in solution:
        route_distance = 0
        # Calcula a distância total da rota incluindo a cidade base no início e no final
        for i in range(len(route) - 1):
            route_distance += distance_matrix[route[i]][route[i+1]]
        total_distance += route_distance
    return total_distance

Seleção por torneio

In [860]:
def tournament_selection(population, fitnesses):
    selected = random.sample(list(zip(population, fitnesses)), 3)
    selected.sort(key=lambda x: x[1])
    return selected[0][0]

Operador de cruzamento (crossover)

In [861]:
def crossover(parent1, parent2):
    probability = random.random()
    if probability > 0.7:
        return random.choice([parent1, parent2])
    
    # Realiza o crossover (Order Crossover - OX)
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [-1] * size
    child[start:end] = parent1[start:end]
    pointer = 0
    for gene in parent2:
        if gene not in child:
            while child[pointer] != -1:
                pointer += 1
            child[pointer] = gene
    return child

Operador de mutação

In [862]:
def mutate(route, mutation_rate):
    for i in range(len(route)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(route) - 1)
            route[i], route[j] = route[j], route[i]
    return route

Algoritmo Genético

In [863]:
def split_routes_by_demand(route, demands, truck_capacity, base_city):
    routes = []
    current_load = 0
    sub_route = [base_city]  # O caminhão sempre começa na cidade base
    
    for city in route:
        demand = demands[city]  # Demanda da cidade (lembrando que a primeira cidade é a base e não tem demanda)
        if current_load + demand <= truck_capacity:
            sub_route.append(city)
            current_load += demand
        else:
            # Quando o caminhão atinge a capacidade máxima, ele volta à base e começa uma nova viagem
            sub_route.append(base_city)
            routes.append(sub_route)
            sub_route = [base_city, city]  # Nova viagem
            current_load = demand

    sub_route.append(base_city)
    routes.append(sub_route)  # Adiciona a última rota
    return routes

def genetic_algorithm(num_cities, population_size, generations, mutation_rate, truck_capacity, base_city, demands):
    distance_matrix = create_distance_matrix()

    # População inicial: cada solução é uma lista de rotas
    population = [random.sample(range(1, num_cities), num_cities - 1) for _ in range(population_size)]
    best_route = None
    best_fitness = float('inf')
    
    average_fitnesses = []
    worst_fitnesses = []
    best_fitnesses = []

    for generation in range(generations):
        # Cada indivíduo será dividido em múltiplas viagens, considerando a demanda das cidades e a capacidade do caminhão
        population_with_routes = [split_routes_by_demand(route, demands, truck_capacity, base_city) for route in population]
        
        # Avaliando o fitness de cada solução
        fitnesses = [calculate_fitness(solution, distance_matrix, base_city) for solution in population_with_routes]
        
        # Armazenando as estatísticas
        average_fitness = np.mean(fitnesses)
        worst_fitness = max(fitnesses)
        current_best_fitness = min(fitnesses)
        
        average_fitnesses.append(average_fitness)
        worst_fitnesses.append(worst_fitness)
        best_fitnesses.append(current_best_fitness)
        
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            best_route = population_with_routes[fitnesses.index(current_best_fitness)]

        new_population = []
        for _ in range(population_size):
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            new_population.append(child)

        population = new_population
        
    statistics_array = []
    fitness_over_time = []
    for generation in range(generations):
        statistics = {
            'generation': generation,
            'mean_fitness': average_fitnesses[generation],
            'worst_fitness': worst_fitnesses[generation],
            'best_fitness': best_fitnesses[generation]
        }
        statistics_array.append(statistics)
        fitness_over_time.append([best_fitnesses[generation], generation])
    
    with open('statistics.json', 'w') as file:
        json.dump(statistics_array, file, indent=4) 
    with open('fitness_over_time.json', 'w') as file:
        json.dump(fitness_over_time, file, indent=4)
        
    return best_route, best_fitness

    '''
    # Plotando os gráficos
    plt.figure(figsize=(10, 6))
    plt.plot(best_fitnesses, label='Melhor Fitness')
    plt.plot(worst_fitnesses, label='Pior Fitness')
    plt.plot(average_fitnesses, label='Média do Fitness')
    plt.xlabel('Gerações')
    plt.ylabel('Fitness')
    plt.title('Evolução do Fitness ao Longo das Gerações')
    plt.legend()
    plt.show()
    '''

Parâmetros do algoritmo

In [864]:
# Parâmetros do algoritmo
num_cities = 35  # Inclui a cidade base
population_size = 200
generations = 100
mutation_rate = 0.01
base_city = 0  # Cidade de origem dos caminhões
truck_capacity = 1050  # Capacidade do caminhão em termos de urnas
demands = [0, 19, 925, 94, 92, 22, 9, 1019, 89, 15, 203, 59, 21, 20, 33, 10, 40, 144, 25, 77, 59, 216, 92, 15, 572, 22, 18, 324, 365, 56, 28, 73, 284, 14, 160]

print(len(demands))

# Executando o algoritmo
start_time = time.time()
best_route, best_fitness = genetic_algorithm(num_cities, population_size, generations, mutation_rate, truck_capacity, base_city, demands)
end_time = time.time()
execution_time = end_time - start_time

print(f"\nMelhor rota: {best_route}\n")
print(f"Melhor Resultado: {best_fitness}\n")
# Round the execution time to 2 decimal places
execution_time = round(execution_time, 2)
print(f"Tempo de execução: {execution_time} segundos\n")

35

Melhor rota: [[0, 25, 21, 10, 12, 18, 31, 11, 19, 13, 3, 26, 16, 29, 23, 0], [0, 17, 5, 4, 15, 0], [0, 2, 9, 6, 0], [0, 28, 8, 27, 33, 34, 14, 30, 0], [0, 7, 1, 0], [0, 32, 20, 24, 22, 0]]

Melhor Resultado: 803.7946344299361

Tempo de execução: 0.6 segundos

